In [1]:
!nvidia-smi

Wed Sep 11 00:02:24 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.81                 Driver Version: 560.81         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4070      WDDM  |   00000000:01:00.0  On |                  N/A |
| 56%   36C    P5             14W /  215W |    1050MiB /  12282MiB |     21%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import tensorflow as tf
from helper_functions import unzip_data, create_tensorboard_callback,plot_loss_curves,compare_historys

In [3]:
unzip_data('nlp-getting-started.zip')

In [4]:
import pandas as pd

In [5]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
train_df.head()
train_df["text"][1]


'Forest fire near La Ronge Sask. Canada'

In [6]:
train_df_shuffled = train_df.sample(frac=1,random_state=42)

In [7]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [8]:
train_df.target.value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

In [9]:
len(train_df),len(test_df)

(7613, 3263)

In [10]:
import random
random_index = random.randint(0,len(train_df)-5)
for row in train_df_shuffled[["text","target"]][random_index:random_index+5].itertuples():
    _,text,target = row
    print(f"Target:{target}","(real disaster)" if target>0 else "(not rela disaster)")
    print(f"Text :{text}")
    print('\n')
    

Target:0 (not rela disaster)
Text :Thinking about getting a demo car with a friend and joining the demolition derby in kenosha


Target:1 (real disaster)
Text :Police Officer Wounded Suspect Dead After Exchanging Shots: Richmond police officer wounded suspect killed a... http://t.co/w0r8EAOKA0


Target:0 (not rela disaster)
Text :Don't be the cause of your own self destruction


Target:1 (real disaster)
Text :#hot  Funtenna: hijacking computers to send data as sound waves [Black Hat 2015] http://t.co/UMgD92wLjA #prebreak #best


Target:1 (real disaster)
Text :Cross-border terrorism: Pakistan caught red-handed again - The Times of India http://t.co/uiqsfgZoOx




In [11]:
from sklearn.model_selection import train_test_split

train_sentences,val_sentences,train_labels,val_labels=train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                       train_df_shuffled["target"].to_numpy(),
                                                                       test_size = 0.1,
                                                                       random_state=42)


In [12]:
train_sentences[:10],train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

In [13]:
train_sentences

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       ...,
       'Near them on the sand half sunk a shattered visage lies... http://t.co/0kCCG1BT06',
       "kesabaran membuahkan hasil indah pada saat tepat! life isn't about waiting for the storm to pass it's about learning to dance in the rain.",
       "@ScottDPierce @billharris_tv @HarrisGle @Beezersun I'm forfeiting this years fantasy football pool out of fear I may win n get my ass kicked"],
      dtype=object)

In [14]:
from tensorflow.keras.layers import TextVectorization

In [15]:
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [16]:
max_vocab_length = 10000
max_length = 15

text_vectorizer = TextVectorization(
    max_tokens=max_vocab_length,
    standardize="lower_and_strip_punctuation",
    split = "whitespace",
    ngrams= None,
    output_mode = 'int',
    output_sequence_length = max_length,
    pad_to_max_tokens = True,
)


In [17]:
text_vectorizer.adapt(train_sentences)

In [18]:
sample_sentence = "hello world"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[1400,  107,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0]], dtype=int64)>

In [19]:
random_sentence = random.choice(train_sentences)
print(random_sentence)
text_vectorizer(random_sentence)

I can't believe @myfriendmina photo bombed a screenshot


<tf.Tensor: shape=(15,), dtype=int64, numpy=
array([   8,   98,  610,    1,  343,  467,    3, 8731,    0,    0,    0,
          0,    0,    0,    0], dtype=int64)>

In [20]:
words_in_vocabulary =  text_vectorizer.get_vocabulary()
top_5_words = words_in_vocabulary[:5]
bottom_5_words = words_in_vocabulary[-5:]
len(words_in_vocabulary),top_5_words,bottom_5_words

(10000,
 ['', '[UNK]', 'the', 'a', 'in'],
 ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1'])

In [53]:
# from tensorflow.keras import layers

tf.random.set_seed(42)
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim=128, # set size of embedding vector
                             embeddings_initializer="uniform", # default, intialize randomly
                             input_length=max_length, # how long is each input
                             name="embedding_1") 

embedding

In [38]:
embeddings

In [39]:
random_sentence = random.choice(train_sentences)
print(random_sentence)
print("\n")
print("embedded version")
embeddings(text_vectorizer([random_sentence]))

Set a new record.... 7 states in 4 days. I don't even know where I am when I wake up anymore. What a whirlwind! Loving every minute though.


embedded version


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.03607414, -0.01530086,  0.03741441, ...,  0.0014539 ,
         -0.01282183,  0.02818838],
        [ 0.0117584 ,  0.04194099,  0.00546082, ...,  0.03996438,
         -0.01849579, -0.03030136],
        [-0.00469812, -0.02071757, -0.04782669, ...,  0.03949722,
          0.04620988, -0.04793452],
        ...,
        [ 0.03388179, -0.01566985,  0.03907767, ...,  0.00290481,
          0.03598301, -0.00936996],
        [-0.02061365, -0.03927447,  0.03179053, ...,  0.01805217,
          0.04069645, -0.01265464],
        [ 0.01898584, -0.0190922 ,  0.04490279, ..., -0.02289447,
         -0.02166978, -0.03533771]]], dtype=float32)>

In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0 = Pipeline([
    ("tfidf",TfidfVectorizer()),
    ("clf",MultinomialNB()),
])

model_0.fit(train_sentences,train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [41]:
baseline_score=model_0.score(val_sentences,val_labels)
baseline_score

0.7926509186351706

In [42]:
baseline_preds=model_0.predict(val_sentences)

In [43]:
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1],
      dtype=int64)

In [44]:
from sklearn.metrics import accuracy_score,precision_recall_fscore_support

In [45]:
def calculate_results(y_true,y_pred):
    model_accuaracy = accuracy_score(y_true,y_pred) *100
    model_precision,model_recall,model_f1,_=precision_recall_fscore_support(y_true,y_pred,average="weighted")
    model_results={
        "accuracy":model_accuaracy,
        "precision":model_precision,
        "recall":model_recall,
        "f1":model_f1
    }
    return model_results

In [46]:
baseline_results=calculate_results(val_labels,baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [47]:
from helper_functions import create_tensorboard_callback

In [48]:
SAVE_DIR='model_logs'

In [54]:
from tensorflow.keras import  layers
inputs=layers.Input(shape=(1,),dtype=tf.string)
x=text_vectorizer(inputs)
x=embeddings(x)
x = layers.GlobalAveragePooling1D()(x) # lower the dimensionality of the embedding (try running the model without this layer and see what happens)
outputs = layers.Dense(1,activation="sigmoid")(x)
model_1 = tf.keras.Model(inputs,outputs,name="model_1_dense")

In [55]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense_2 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [56]:
model_1.compile(
    loss='binary_crossentropy',
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy']
)

In [57]:
model_1.history = model_1.fit(x=train_sentences,
                              y=train_labels,
                              epochs=5,
                              validation_data=(val_sentences,val_labels),callbacks=[create_tensorboard_callback(SAVE_DIR,'model_1_dense')])

Saving TensorBoard log files to: model_logs/model_1_dense/20240911-000535
Epoch 1/5
215/215 [==============================] - 1s 3ms/step - loss: 0.6099 - accuracy: 0.6898 - val_loss: 0.5370 - val_accuracy: 0.7533
Epoch 2/5
215/215 [==============================] - 0s 2ms/step - loss: 0.4421 - accuracy: 0.8186 - val_loss: 0.4697 - val_accuracy: 0.7874
Epoch 3/5
215/215 [==============================] - 1s 2ms/step - loss: 0.3473 - accuracy: 0.8600 - val_loss: 0.4593 - val_accuracy: 0.7887
Epoch 4/5
215/215 [==============================] - 1s 2ms/step - loss: 0.2857 - accuracy: 0.8924 - val_loss: 0.4640 - val_accuracy: 0.7940
Epoch 5/5
215/215 [==============================] - 1s 3ms/step - loss: 0.2388 - accuracy: 0.9127 - val_loss: 0.4763 - val_accuracy: 0.7874
